In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
import pydotplus
import pydot
import matplotlib.pyplot as plt
from tensorflow.keras import layers

In [ ]:

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
import keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Input, Dense, Flatten, LSTM, GRU, Conv1D,Conv2D, MaxPooling1D, Dropout, Activation, Concatenate

In [ ]:
path = os.getcwd()
all_files = glob.glob(os.path.join(path, "*.txt"))
df_from_each_file = [ pd.read_csv(file,sep="\t", header=None) for file in all_files]
df = pd.concat(df_from_each_file, ignore_index=True)
df.columns = ["Sr. No.", "Tweet", "Class","Intensity"]
df=df.sample(frac=1)
pd.set_option('display.max_colwidth', -1)

In [ ]:
df[df['Class']=='fear'].sample(5)

In [ ]:
import preprocessor as p
from gensim.parsing.preprocessing import *

def preprocess_tweet(row):
    text = row['Tweet']
    text = p.clean(text)
    return text

def stopword_removal(row):
    text = row['Tweet']
    text = remove_stopwords(text)
    return text

def lower_casing(row):
    text = row['Tweet']
    text = text.lower().replace('[^\w\s]',' ').replace('\s\s+', ' ')
    return text

def keep_aplhaNumerics_Only(row):
    text = row['Tweet']
    text = strip_non_alphanum(text)
    return text

def preprocessData(df):
    df['Tweet'] = df.apply(preprocess_tweet, axis=1)
    df['Tweet'] = df.apply(stopword_removal, axis=1)
    df['Tweet'] = df.apply(keep_aplhaNumerics_Only, axis=1)
    df['Tweet'] = df.apply(lower_casing, axis=1)
    X = df['Tweet']
    Y = df[['Class','Intensity']]
    return X.to_numpy(),Y

In [ ]:
X_train, Y_train = preprocessData(df)
tokenizer = Tokenizer(num_words= 200000) 
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
X_train_encoded = pad_sequences(sequences, maxlen=30)

In [ ]:
Y1_train = Y_train['Class']
Y2_train = Y_train['Intensity']
le = LabelEncoder()

Y1_train_encoded = le.fit_transform(Y1_train)
Y1_train_encoded = to_categorical(Y1_train_encoded)
Y2_train_encoded = Y2_train

In [ ]:
Y1_train = le.inverse_transform(Y1_train)
word_index = tokenizer.word_index

In [ ]:
embeddings_index = {}
with open("glove.6B/glove.6B.300d.txt",encoding="utf8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

In [ ]:
VOCAB_SIZE = len(tokenizer.word_index) + 2
EMBEDDING_DIM = 300
hits = 0
misses = 0


embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

### LSTM

In [ ]:
def LSTM_model(task):
    model = Sequential()
    model.add(Embedding(VOCAB_SIZE, EMBEDDING_DIM, weights = [embedding_matrix], trainable=True, input_length=X_train_encoded.shape[1]))
    #model.add(Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=X_train_encoded.shape[1]))
    model.add(LSTM(128,activation='relu',return_sequences=True))
    model.add(LSTM(128,activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(128,activation='relu'))
    model.add(Dense(100,activation='relu'))

    if task == "Classification":
        model.add(Dense(4, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    elif task =="Regression":
        model.add(Dense(1,activation='sigmoid'))
        model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mse',pearson_r])
    else:
        print("ERROR: INVALID TASK")
        return

    return model

### GRU

In [ ]:
def GRU_model(task):
    model = Sequential()
    model.add(Embedding(VOCAB_SIZE, EMBEDDING_DIM, weights = [embedding_matrix], trainable=True, input_length=X_train_encoded.shape[1]))
    #model.add(Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=X_train_encoded.shape[1]))
    model.add(GRU(128,activation='relu',return_sequences=True))
    model.add(GRU(128,activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(128,activation='relu'))
    model.add(Dense(100,activation='relu'))

    if task == "Classification":
        model.add(Dense(4, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    elif task =="Regression":
        model.add(Dense(1,activation='sigmoid'))
        model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mse',pearson_r])
    else:
        print("ERROR: INVALID TASK")
        return

    return model

### CNN

In [ ]:
def CNN_model(task):
    Input_layer     = Input(shape = X_train_encoded.shape[1], name="Input_layer")
    Embedding_layer = Embedding(VOCAB_SIZE, EMBEDDING_DIM,weights = [embedding_matrix],trainable=True, input_length=X_train_encoded.shape[1], name="Embedding_layer")(Input_layer)
    #Embedding_layer = Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=X_train_encoded.shape[1])(Input_layer)
    CNN1_layer1     = Conv1D(100, kernel_size=2, padding="same", activation="relu")(Embedding_layer)
    CNN1_layer2     = Conv1D(100, kernel_size=3, padding="same", activation="relu")(Embedding_layer)
    CNN1_layer3     = Conv1D(100, kernel_size=4, padding="same", activation="relu")(Embedding_layer)
    Concat_layer1   = Concatenate(axis=1)([CNN1_layer1, CNN1_layer2, CNN1_layer3])
    Pool_layer1     = MaxPooling1D(pool_size=2)(Concat_layer1)
    CNN2_layer1     = Conv1D(100, kernel_size=2, padding="same", activation="relu")(Pool_layer1)
    CNN2_layer2     = Conv1D(100, kernel_size=3, padding="same", activation="relu")(Pool_layer1)
    CNN2_layer3     = Conv1D(100, kernel_size=4, padding="same", activation="relu")(Pool_layer1)
    Concat_layer2   = Concatenate(axis=1)([CNN2_layer1, CNN2_layer2, CNN2_layer3])
    Pool_layer2     = MaxPooling1D(pool_size=2)(Concat_layer2)
    Flatten_layer   = Flatten()(Pool_layer2)
    Dense_layer1    = Dense(128,activation='relu', name="Dense_layer1a")(Flatten_layer)
    Dense_layer2    = Dense(100,activation='relu', name="Dense_layer1b")(Dense_layer1)

    if task == "Classification":
        predictions_task1 = Dense(4, activation='softmax', name="predictions_task1")(Dense_layer2)
        model= tf.keras.models.Model(Input_layer,predictions_task1)
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    elif task =="Regression":
        predictions_task2 = Dense(1, activation='sigmoid', name="predictions_task2")(Dense_layer2)
        model= tf.keras.models.Model(Input_layer,predictions_task2)
        model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mse',pearson_r])
    else:
        print("ERROR: INVALID TASK")
        return

    return model

In [ ]:
def getAccuracy(model, X, Y):
    history = model.fit(X,Y, epochs=40, batch_size=64,validation_split=0.1,
                        callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    return history.history['accuracy'],history.history['val_accuracy']

In [ ]:
def getPearsonR(model, X, Y):
    history = model.fit(X,Y, epochs=40, batch_size=64,validation_split=0.1,
                        callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

    # summarize history for accuracy
    plt.plot(history.history['pearson_r'])
    plt.plot(history.history['val_pearson_r'])
    plt.title('Model Pearsn R Coefficient')
    plt.ylabel('pearson_r')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    return history.history['pearson_r'],history.history['val_pearson_r']

In [ ]:
task1 = "Classification"
task2 = "Regression"

Results_Task1=dict()
Results_Task2=dict()

In [ ]:
print("LSTM Model\n\n")
acc, val_acc = getAccuracy(LSTM_model(task1), X_train_encoded, Y1_train_encoded)
Results_Task1['LSTM']={'Accuracy':acc[-1],'Val_Accuracy':val_acc[-1]}

In [ ]:
print("GRU Model\n\n")
acc, val_acc = getAccuracy(GRU_model(task1), X_train_encoded, Y1_train_encoded)
Results_Task1['GRU']={'Accuracy':acc[-1],'Val_Accuracy':val_acc[-1]}

In [ ]:
print("CNN Model\n\n")
acc, val_acc = getAccuracy(CNN_model(task1), X_train_encoded, Y1_train_encoded)
Results_Task1['CNN']={'Accuracy':acc[-1],'Val_Accuracy':val_acc[-1]}

In [ ]:
from keras import backend as K
def pearson_r(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x, axis=0)
    my = K.mean(y, axis=0)
    xm, ym = x - mx, y - my
    r_num = K.sum(xm * ym)
    x_square_sum = K.sum(xm * xm)
    y_square_sum = K.sum(ym * ym)
    r_den = K.sqrt(x_square_sum * y_square_sum)
    r = r_num / r_den
    return K.mean(r)

In [ ]:
print("LSTM Model\n\n")
pearson_r, val_pearson_r = getPearsonR(LSTM_model(task2), X_train_encoded, Y2_train_encoded)
Results_Task2['LSTM']={'pearson_r':pearson_r[-1],'Val_pearson_r':val_pearson_r[-1]}

In [ ]:
print("GRU Model\n\n")
pearson_r, val_pearson_r = getPearsonR(GRU_model(task2), X_train_encoded, Y2_train_encoded)
Results_Task2['GRU']={'pearson_r':pearson_r[-1],'Val_pearson_r':val_pearson_r[-1]}

In [ ]:
print("CNN Model\n\n")
pearson_r, val_pearson_r = getPearsonR(CNN_model(task2), X_train_encoded, Y2_train_encoded)
Results_Task2['CNN']={'pearson_r':pearson_r[-1],'Val_pearson_r':val_pearson_r[-1]}

In [ ]:
print("Single Task Framework - Task 1 Results\n")
# Print the names of the column
print ("{:<10} {:<15} {:<10}".format('Model', 'Train Accuracy', ' Test Accuracy'))

# print each data item.
for key, value in Results_Task1.items():
    train_acc = round(value['Accuracy'],3)
    val_acc   = round(value['Val_Accuracy'],3)
    print ("{:<10} {:<17} {:<10}".format(key, train_acc, val_acc))

In [ ]:
print("Single Task Framework - Task 2 Results\n")
print ("{:<10} {:<15} {:<10}".format('Model', 'Train pearson_r', ' Val pearson_r'))

for key, value in Results_Task2.items():
    train_acc = round(value['pearson_r'],3)
    val_acc   = round(value['Val_pearson_r'],3)
    print ("{:<10} {:<17} {:<10}".format(key, train_acc, val_acc))

#
# __Multi-tasking Framework__

In [ ]:
Multitask_Results_Task1 = {}
Multitask_Results_Task2 = {}
alpha = 0.185

In [ ]:
from keras import backend as K
def pearson_r(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x, axis=0)
    my = K.mean(y, axis=0)
    xm, ym = x - mx, y - my
    r_num = K.sum(xm * ym)
    x_square_sum = K.sum(xm * xm)
    y_square_sum = K.sum(ym * ym)
    r_den = K.sqrt(x_square_sum * y_square_sum)
    r = r_num / r_den
    return K.mean(r)

In [ ]:
def create_multihead_LSTM_model(X):
    #Shared Layers
    Input_layer       = Input(shape = X.shape[1], name="Input_layer")
    Embedding_layer   = Embedding(VOCAB_SIZE, EMBEDDING_DIM,weights = [embedding_matrix], trainable=True, input_length=X.shape[1], name="Embedding_layer")(Input_layer)
    #Embedding_layer  = Embedding(VOCAB_SIZE,EMBEDDING_DIM, input_length=X.shape[1], name="Embedding_layer")(Input_layer)
    LSTM_layer1       = LSTM(128,activation='relu', return_sequences=True, name="LSTM_layer1" )(Embedding_layer)
    LSTM_layer2       = LSTM(128,activation='relu', name="LSTM_layer2")(LSTM_layer1)
    Dropout_layer     = Dropout(0.25, name="Dropout_layer")(LSTM_layer2)

 
    Dense_layer1a     = Dense(128,activation='relu', name="Dense_layer1a")(Dropout_layer)
    Dense_layer1b     = Dense(100,activation='relu', name="Dense_layer1b")(Dense_layer1a)

  
    Dense_layer2a     = Dense(128,activation='relu', name="Dense_layer2a")(Dropout_layer)
    Dense_layer2b     = Dense(100,activation='relu', name="Dense_layer2b")(Dense_layer2a)

  
    predictions_task1 = Dense(4, activation='softmax', name="predictions_task1")(Dense_layer1b)
    predictions_task2 = Dense(1, name="predictions_task2")(Dense_layer2b)

    model= tf.keras.models.Model(Input_layer, [predictions_task1,predictions_task2])
    model.compile( loss= {'predictions_task1':'categorical_crossentropy' , 'predictions_task2':'mean_absolute_error' },
                   loss_weights=[alpha,1-alpha],
                   optimizer='adam',
                   metrics={'predictions_task1':'accuracy' , 'predictions_task2':pearson_r },
                 )
    return model

In [ ]:
Multihead_LSTM_model = create_multihead_LSTM_model(X_train_encoded)

In [ ]:
Multihead_LSTM_model_history = Multihead_LSTM_model.fit(X_train_encoded, [Y1_train_encoded,Y2_train_encoded], epochs=40, batch_size=64,validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

In [ ]:
plt.plot(Multihead_LSTM_model_history.history['predictions_task1_accuracy'])
plt.plot(Multihead_LSTM_model_history.history['val_predictions_task1_accuracy'])
plt.title('Multitasking LSTM Model Accuracy Task 1')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:

plt.plot(Multihead_LSTM_model_history.history['predictions_task2_pearson_r'])
plt.plot(Multihead_LSTM_model_history.history['val_predictions_task2_pearson_r'])
plt.title('Multitasking LSTM Model Pearson R Task 2')
plt.ylabel(' Pearson R')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(Multihead_LSTM_model_history.history['predictions_task2_loss'])
plt.plot(Multihead_LSTM_model_history.history['val_predictions_task2_loss'])
plt.title('Multitasking LSTM Model Loss Task 2')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
Multitask_Results_Task1['LSTM'] ={'Accuracy':Multihead_LSTM_model_history.history['predictions_task1_accuracy'][-1],
                                  'Val_Accuracy':Multihead_LSTM_model_history.history['val_predictions_task1_accuracy'][-1]
                                 }
Multitask_Results_Task2['LSTM'] ={'pearson_r':Multihead_LSTM_model_history.history['predictions_task2_pearson_r'][-1],
                                  'Val_pearson_r':Multihead_LSTM_model_history.history['val_predictions_task2_pearson_r'][-1]}

### Create Model

In [ ]:
def create_multihead_GRU_model(X):
    #Shared Layers
    Input_layer       = Input(shape = X.shape[1], name="Input_layer")
    Embedding_layer   = Embedding(VOCAB_SIZE, EMBEDDING_DIM,weights = [embedding_matrix], trainable=True, input_length=X.shape[1], name="Embedding_layer")(Input_layer)
    #Embedding_layer   = Embedding(VOCAB_SIZE,EMBEDDING_DIM, input_length=X.shape[1], name="Embedding_layer")(Input_layer)
    GRU_layer1        = GRU(128,activation='relu', return_sequences=True, name="GRU_layer1" )(Embedding_layer)
    GRU_layer2        = GRU(128,activation='relu', name="GRU_layer2")(GRU_layer1)
    Dropout_layer     = Dropout(0.25, name="Dropout_layer")(GRU_layer2)

    #Task Specific Layer for Task 1
    Dense_layer1a     = Dense(128,activation='relu', name="Dense_layer1a")(Dropout_layer)
    Dense_layer1b     = Dense(100,activation='relu', name="Dense_layer1b")(Dense_layer1a)

    #Task Specific Layer for Task 2
    Dense_layer2a     = Dense(128,activation='relu', name="Dense_layer2a")(Dropout_layer)
    Dense_layer2b     = Dense(100,activation='relu', name="Dense_layer2b")(Dense_layer2a)

    # Predictions for each task
    predictions_task1 = Dense(4, activation='softmax', name="predictions_task1")(Dense_layer1b)
    predictions_task2 = Dense(1, activation='sigmoid', name="predictions_task2")(Dense_layer2b)

    model= tf.keras.models.Model(Input_layer, [predictions_task1,predictions_task2])
    model.compile( loss= {'predictions_task1':'categorical_crossentropy' , 'predictions_task2':'mean_absolute_error' },
                   loss_weights=[alpha,1-alpha],
                   optimizer='adam',
                   metrics={'predictions_task1':'accuracy' , 'predictions_task2':pearson_r },
                 )
    return model

In [ ]:
Multihead_GRU_model = create_multihead_GRU_model(X_train_encoded)

### Train Model

In [ ]:
Multihead_GRU_model_history = Multihead_GRU_model.fit(X_train_encoded, [Y1_train_encoded,Y2_train_encoded], epochs=40, batch_size=64,validation_split=0.1,
                     callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

### Summarize Model Accuracy

In [ ]:
# Summarize history for Task 1
plt.plot(Multihead_GRU_model_history.history['predictions_task1_accuracy'])
plt.plot(Multihead_GRU_model_history.history['val_predictions_task1_accuracy'])
plt.title('Multitasking Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# Summarize history for Task 2
plt.plot(Multihead_GRU_model_history.history['predictions_task2_pearson_r'])
plt.plot(Multihead_GRU_model_history.history['val_predictions_task2_pearson_r'])
plt.title('Multitasking Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
Multitask_Results_Task1['GRU'] ={'Accuracy':Multihead_GRU_model_history.history['predictions_task1_accuracy'][-1],
                                  'Val_Accuracy':Multihead_GRU_model_history.history['val_predictions_task1_accuracy'][-1]}
Multitask_Results_Task2['GRU'] ={'pearson_r':Multihead_GRU_model_history.history['predictions_task2_pearson_r'][-1],
                                  'Val_pearson_r':Multihead_GRU_model_history.history['val_predictions_task2_pearson_r'][-1]}

### Plotting Model Architecture

## __Multi-tasking CNN Model__

### Create Model

In [ ]:
def create_multihead_CNN_model(X):
    #Shared Layers
    Input_layer       = Input(shape = X.shape[1], name="Input_layer")
    Embedding_layer   = Embedding(VOCAB_SIZE, EMBEDDING_DIM,weights = [embedding_matrix], trainable=True, input_length=X.shape[1], name="Embedding_layer")(Input_layer)
    #Embedding_layer  = Embedding(VOCAB_SIZE,EMBEDDING_DIM, input_length=X.shape[1], name="Embedding_layer")(Input_layer)
    CNN1_layer1       = Conv1D(100, kernel_size=2, padding="same", activation="relu")(Embedding_layer)
    CNN1_layer2       = Conv1D(100, kernel_size=3, padding="same", activation="relu")(Embedding_layer)
    CNN1_layer3       = Conv1D(100, kernel_size=4, padding="same", activation="relu")(Embedding_layer)
    Concat_layer1     = Concatenate(axis=1)([CNN1_layer1, CNN1_layer2, CNN1_layer3])
    Pool_layer1       = MaxPooling1D(pool_size=2)(Concat_layer1)
    CNN2_layer1       = Conv1D(100, kernel_size=2, padding="same", activation="relu")(Pool_layer1)
    CNN2_layer2       = Conv1D(100, kernel_size=3, padding="same", activation="relu")(Pool_layer1)
    CNN2_layer3       = Conv1D(100, kernel_size=4, padding="same", activation="relu")(Pool_layer1)
    Concat_layer2     = Concatenate(axis=1)([CNN2_layer1, CNN2_layer2, CNN2_layer3])
    Pool_layer2       = MaxPooling1D(pool_size=2)(Concat_layer2)
    Flatten_layer     = Flatten()(Pool_layer2)

    #Task Specific Layer for Task 1
    Dense_layer1a     = Dense(128,activation='relu', name="Dense_layer1a")(Flatten_layer)
    Dense_layer1b     = Dense(100,activation='relu', name="Dense_layer1b")(Dense_layer1a)

    #Task Specific Layer for Task 2
    Dense_layer2a     = Dense(128,activation='relu', name="Dense_layer2a")(Flatten_layer)
    Dense_layer2b     = Dense(100,activation='relu', name="Dense_layer2b")(Dense_layer2a)

    # Predictions for each task
    predictions_task1 = Dense(4, activation='softmax', name="predictions_task1")(Dense_layer1b)
    predictions_task2 = Dense(1, activation='sigmoid', name="predictions_task2")(Dense_layer2b)

    model= tf.keras.models.Model(Input_layer, [predictions_task1,predictions_task2])
    model.compile( loss= {'predictions_task1':'categorical_crossentropy' , 'predictions_task2':'mean_absolute_error' },
                   optimizer='adam',
                   loss_weights=[alpha,1-alpha],
                   metrics={'predictions_task1':'accuracy' , 'predictions_task2':pearson_r },
                 )
    return model

In [ ]:
Multihead_CNN_model = create_multihead_CNN_model(X_train_encoded)

In [ ]:
Multihead_CNN_model_history = Multihead_CNN_model.fit(X_train_encoded, [Y1_train_encoded,Y2_train_encoded], epochs=40, batch_size=64,validation_split=0.1,
                     callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

### Summarize Model Accuracy

In [ ]:
# Summarize history for Task 1
plt.plot(Multihead_CNN_model_history.history['predictions_task1_accuracy'])
plt.plot(Multihead_CNN_model_history.history['val_predictions_task1_accuracy'])
plt.title('Multitasking Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# Summarize history for Task 2
plt.plot(Multihead_CNN_model_history.history['predictions_task2_pearson_r'])
plt.plot(Multihead_CNN_model_history.history['val_predictions_task2_pearson_r'])
plt.title('Multitasking Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
Multitask_Results_Task1['CNN'] ={'Accuracy':Multihead_CNN_model_history.history['predictions_task1_accuracy'][-1],
                                  'Val_Accuracy':Multihead_CNN_model_history.history['val_predictions_task1_accuracy'][-1]}
Multitask_Results_Task2['CNN'] ={'pearson_r':Multihead_CNN_model_history.history['predictions_task2_pearson_r'][-1],
                                  'Val_pearson_r':Multihead_CNN_model_history.history['val_predictions_task2_pearson_r'][-1]}

### Plotting Model Architecture

## Performance Comparision

### Task 1

In [ ]:
print("Multitasking Framework Task 1 Results\n")
# Print the names of the column
print ("{:<10} {:<15} {:<10}".format('Model', 'Train Accuracy', ' Val Accuracy'))

# print each data item.
for key, value in Multitask_Results_Task1.items():
    train_acc = round(value['Accuracy'],3)
    val_acc   = round(value['Val_Accuracy'],3)
    print ("{:<10} {:<17} {:<10}".format(key, train_acc, val_acc))

### Task 2

In [ ]:
print("Multitasking Framework Task 2 Results\n")
# Print the names of the column
print ("{:<10} {:<15} {:<10}".format('Model', 'Train Pearson_R', ' Val Pearson_R'))

# print each data item.
for key, value in Multitask_Results_Task2.items():
    train_acc = round(value['pearson_r'],3)
    val_acc   = round(value['Val_pearson_r'],3)
    print ("{:<10} {:<17} {:<10}".format(key, train_acc, val_acc))

# __Performance Comparision : Single Task Vs Multi Task Framework__

### Task 1

In [ ]:
print("Performance Comparision : Single Task Vs Multi Task Framework\n\n")
# Print the names of the column

print ("{:<20} {:<30} {:<20}".format('Framework','Single Task', 'Multi Task'))
print ("{:<10} {:<15} {:<10} {:<15} {:<10}".format('Model', 'Train Accuracy', ' Val Accuracy  ','  Train Accuracy', ' Val Accuracy'))

# print each data item.
for key in ['LSTM','GRU','CNN']:
    train_acc1 = round(Results_Task1[key]['Accuracy'],3)
    val_acc1   = round(Results_Task1[key]['Val_Accuracy'],3)
    train_acc2 = round(Multitask_Results_Task1[key]['Accuracy'],3)
    val_acc2   = round(Multitask_Results_Task1[key]['Val_Accuracy'],3)
    print ("{:<10} {:<17} {:<20} {:<17} {:<10}".format(key, train_acc1, val_acc1,train_acc2, val_acc2))

### Task 2

In [ ]:
print("Performance Comparision : Single Task Vs Multi Task Framework\n\n")
# Print the names of the column

print ("{:<20} {:<30} {:<20}".format('Framework','Single Task', 'Multi Task'))
print ("{:<10} {:<15} {:<10} {:<15} {:<10}".format('Model', 'Train pearson_r', ' Val pearson_r  ','  Train pearson_r', ' Val pearson_r'))

# print each data item.
for key in ['LSTM','GRU','CNN']:
    train_acc1 = round(Results_Task2[key]['pearson_r'],3)
    val_acc1   = round(Results_Task2[key]['Val_pearson_r'],3)
    train_acc2 = round(Multitask_Results_Task2[key]['pearson_r'],3)
    val_acc2   = round(Multitask_Results_Task2[key]['Val_pearson_r'],3)
    print ("{:<10} {:<17} {:<20} {:<17} {:<10}".format(key, train_acc1, val_acc1,train_acc2, val_acc2))

# __Ensemble Framework__

## Task-aware Representations

In [ ]:
Task1_aware_LSTM_Representation = Multihead_LSTM_model.get_layer("Dense_layer1b")
Task2_aware_LSTM_Representation = Multihead_LSTM_model.get_layer("Dense_layer2b")

Task1_aware_GRU_Representation = Multihead_GRU_model.get_layer("Dense_layer1b")
Task2_aware_GRU_Representation = Multihead_GRU_model.get_layer("Dense_layer2b")

Task1_aware_CNN_Representation = Multihead_CNN_model.get_layer("Dense_layer1b")
Task2_aware_CNN_Representation = Multihead_CNN_model.get_layer("Dense_layer2b")

In [ ]:
def create_ensemble_model(Task_aware_Representations):
    #Shared Layers
    Concat_layer    = Concatenate(axis=1)(Task_aware_Representations)
    Dense_layer1    = Dense(128,activation='relu', name="Dense_layer1")(Concat_layer)
    Dense_layer2    = Dense(128,activation='relu', name="Dense_layer2")(Dense_layer1)

    #Task Specific Layer for Task 1
    Dense_layer_task1 = Dense(128,activation='relu', name="Dense_layer_task1")(Dense_layer2)
    predictions_task1 = Dense(4, activation='softmax', name="predictions_task1")(Dense_layer_task1)

    #Task Specific Layer for Task 1
    Dense_layer_task2 = Dense(128,activation='relu', name="Dense_layer_task2")(Dense_layer2)
    predictions_task2 = Dense(1, activation='sigmoid', name="predictions_task2")(Dense_layer_task2)

    model= tf.keras.models.Model(Input_layer, [predictions_task1,predictions_task2])
    model.compile( loss= {'predictions_task1':'categorical_crossentropy' , 'predictions_task2':'mean_absolute_error' },
                   optimizer='adam',
                   metrics={'predictions_task1':'accuracy' , 'predictions_task2':pearson_r },
               )
    return model

In [ ]:
Multihead_Ensemble_model = create_ensemble_model(Task_aware_Representations)